Based entirely on Abishek's, Bluefool's and Bojan's kernels. 
For no ther reason than to reproduce the results. 
No original contribution whatsoever. 
Didn't do any new stuff other than merging two cells and got LB score 0.399 :P
You are welcome.

Links to orginal kernel https://www.kaggle.com/tunguz/annoying-ab-shreck-and-bluetooth
https://www.kaggle.com/domcastro/let-s-annoy-abhishek
https://www.kaggle.com/abhishek/maybe-something-interesting-here

EAS - All I have done is change the parameters for the LGB. I am also abandoning the OptimizedRounder and have hand-tuned the coefficients to match the distribution of the test predictions to the distribution of the train targets.

** Change Log **

 * v19 - added regularization lambda (0.2)
 * v21 - lowered lambda (0.05)
 * v22 - lowered lambda again (0.02)
 * v23 - lambda up to (0.075)
 * v24 - lambda back to 0.05, commenting out tfidf code that causes error
 * v25 - uncommenting commented out code
 * v26 - fixed bug that was throwing error for tf-idf code
 * v27 - examining code that had been throwing error, fixing the error lowers the score
 * v28 - passing list of categorical features
 * v29 - tuning params
 * v30 - using more tfidf features
 * v31, 32 - playing with tfidf params
 * v35 - back to 120 tfidf svd features
 * v36 - reverting to baseline and not using OptimizedRounder, hand-tuning coefs instead
 
 ** mt-changelog **
 v4 - 
 

In [5]:
import json

import scipy as sp
import pandas as pd
import numpy as np

from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from collections import Counter

import lightgbm as lgb
np.random.seed(369)

In [6]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [7]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))    

## Load Data

In [8]:
print('Train')
train = pd.read_csv(r"../input/petfinder-adoption-prediction/train/train.csv")
print(train.shape)

print('Test')
test = pd.read_csv(r"../input/petfinder-adoption-prediction/test/test.csv")
print(test.shape)

print('Breeds')
breeds = pd.read_csv(r"../input/petfinder-adoption-prediction/breed_labels.csv")
print(breeds.shape)

print('Colors')
colors = pd.read_csv(r"../input/petfinder-adoption-prediction/color_labels.csv")
print(colors.shape)

print('States')
states = pd.read_csv(r"../input/petfinder-adoption-prediction/state_labels.csv")
print(states.shape)

target = train['AdoptionSpeed']
train_id = train['PetID']
test_id = test['PetID']
train.drop(['AdoptionSpeed'], axis=1, inplace=True)
#test.drop(['PetID'], axis=1, inplace=True)

Train
(14993, 24)
Test
(3948, 23)
Breeds
(307, 3)
Colors
(7, 2)
States
(15, 2)


In [9]:
# state GDP: https://en.wikipedia.org/wiki/List_of_Malaysian_states_by_GDP
state_gdp = {
    41336: 116.679,
    41325: 40.596,
    41367: 23.02,
    41401: 190.075,
    41415: 5.984,
    41324: 37.274,
    41332: 42.389,
    41335: 52.452,
    41330: 67.629,
    41380: 5.642,
    41327: 81.284,
    41345: 80.167,
    41342: 121.414,
    41326: 280.698,
    41361: 32.270
}

# state population: https://en.wikipedia.org/wiki/Malaysia
state_population = {
    41336: 33.48283,
    41325: 19.47651,
    41367: 15.39601,
    41401: 16.74621,
    41415: 0.86908,
    41324: 8.21110,
    41332: 10.21064,
    41335: 15.00817,
    41330: 23.52743,
    41380: 2.31541,
    41327: 15.61383,
    41345: 32.06742,
    41342: 24.71140,
    41326: 54.62141,
    41361: 10.35977
}

train["state_gdp"] = train.State.map(state_gdp)
train["state_population"] = train.State.map(state_population)
test["state_gdp"] = test.State.map(state_gdp)
test["state_population"] = test.State.map(state_population)

In [10]:
test.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,state_gdp,state_population
0,1,Puppy,2,307,0,1,1,0,0,2,...,1,150,41326,4475f31553f0170229455e3c5645644f,0,"Puppy is calm for a young dog, but he becomes ...",378fcc4fc,3.0,280.698,54.62141
1,2,London,24,266,0,1,2,7,0,2,...,1,0,41326,4475f31553f0170229455e3c5645644f,0,Urgently seeking adoption. Please contact for ...,73c10e136,1.0,280.698,54.62141
2,2,Snowball,20,266,0,2,7,0,0,2,...,1,150,41326,4475f31553f0170229455e3c5645644f,0,Snowball... doesn't look so good (she is healt...,72000c4c5,1.0,280.698,54.62141
3,2,Malibu,5,266,252,2,1,6,7,2,...,1,100,41326,4475f31553f0170229455e3c5645644f,0,"Malibu: Female, Local Mix, 4-5 months, vaccina...",e147a4b9f,1.0,280.698,54.62141
4,1,Lala Girl,6,307,0,2,1,2,7,2,...,1,150,41326,4475f31553f0170229455e3c5645644f,0,LALA! That's my name. I'm a 6 month old girl d...,43fbba852,1.0,280.698,54.62141


In [11]:
doc_sent_mag = []
doc_sent_score = []
nf_count = 0
for pet in train_id:
    try:
        with open(r'../input/petfinder-adoption-prediction/train_sentiment/' + pet + '.json', 'r') as f:
            sentiment = json.load(f)
        doc_sent_mag.append(sentiment['documentSentiment']['magnitude'])
        doc_sent_score.append(sentiment['documentSentiment']['score'])
    except FileNotFoundError:
        nf_count += 1
        doc_sent_mag.append(-1)
        doc_sent_score.append(-1)

train.loc[:, 'doc_sent_mag'] = doc_sent_mag
train.loc[:, 'doc_sent_score'] = doc_sent_score

doc_sent_mag = []
doc_sent_score = []
nf_count = 0
for pet in test_id:
    try:
        with open(r'../input/petfinder-adoption-prediction/test_sentiment/' + pet + '.json', 'r') as f:
            sentiment = json.load(f)
        doc_sent_mag.append(sentiment['documentSentiment']['magnitude'])
        doc_sent_score.append(sentiment['documentSentiment']['score'])
    except FileNotFoundError:
        nf_count += 1
        doc_sent_mag.append(-1)
        doc_sent_score.append(-1)

test.loc[:, 'doc_sent_mag'] = doc_sent_mag
test.loc[:, 'doc_sent_score'] = doc_sent_score


In [12]:
train_desc = train.Description.fillna("none").values
test_desc = test.Description.fillna("none").values

tfv = TfidfVectorizer(min_df=2,  max_features=None,
        strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        )
    
# Fit TFIDF
tfv.fit(list(train_desc))
X =  tfv.transform(train_desc)
X_test = tfv.transform(test_desc)
print("X (tfidf):", X.shape)

svd = TruncatedSVD(n_components=200)
svd.fit(X)
# print(svd.explained_variance_ratio_.sum())
# print(svd.explained_variance_ratio_)
X = svd.transform(X)
print("X (svd):", X.shape)

# X = pd.DataFrame(X, columns=['svd_{}'.format(i) for i in range(120)])
# train = pd.concat((train, X), axis=1)
# X_test = svd.transform(X_test)
# X_test = pd.DataFrame(X_test, columns=['svd_{}'.format(i) for i in range(120)])
# test = pd.concat((test, X_test), axis=1)

print("train:", train.shape)

X (tfidf): (14993, 181602)
X (svd): (14993, 200)
train: (14993, 27)


In [13]:
## WITHOUT ERROR FIXED
train_desc = train.Description.fillna("none").values
test_desc = test.Description.fillna("none").values

tfv = TfidfVectorizer(min_df=3,  max_features=10000,
        strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        stop_words = 'english')
    
# Fit TFIDF
tfv.fit(list(train_desc))
X =  tfv.transform(train_desc)
X_test = tfv.transform(test_desc)
print("X (tfidf):", X.shape)

svd = TruncatedSVD(n_components=120)
svd.fit(X)
# print(svd.explained_variance_ratio_.sum())
# print(svd.explained_variance_ratio_)
X = svd.transform(X)
print("X (svd):", X.shape)

X = pd.DataFrame(X, columns=['svd_{}'.format(i) for i in range(120)])
train = pd.concat((train, X), axis=1)
X_test = svd.transform(X_test)
X_test = pd.DataFrame(X_test, columns=['svd_{}'.format(i) for i in range(120)])
test = pd.concat((test, X_test), axis=1)

print("train:", train.shape)

X (tfidf): (14993, 10000)
X (svd): (14993, 120)
train: (14993, 147)


In [14]:
vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
nf_count = 0
nl_count = 0
for pet in train_id:
    try:
        with open(r'../input/petfinder-adoption-prediction/train_metadata/' + pet + '-1.json', 'r') as f:
            data = json.load(f)
        vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
        vertex_xs.append(vertex_x)
        vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
        vertex_ys.append(vertex_y)
        bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
        bounding_confidences.append(bounding_confidence)
        bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
        bounding_importance_fracs.append(bounding_importance_frac)
        dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
        dominant_blues.append(dominant_blue)
        dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
        dominant_greens.append(dominant_green)
        dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
        dominant_reds.append(dominant_red)
        dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
        dominant_pixel_fracs.append(dominant_pixel_frac)
        dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
        dominant_scores.append(dominant_score)
        if data.get('labelAnnotations'):
            label_description = data['labelAnnotations'][0]['description']
            label_descriptions.append(label_description)
            label_score = data['labelAnnotations'][0]['score']
            label_scores.append(label_score)
        else:
            nl_count += 1
            label_descriptions.append('nothing')
            label_scores.append(-1)
    except FileNotFoundError:
        nf_count += 1
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_descriptions.append('nothing')
        label_scores.append(-1)

print(nf_count)
print(nl_count)
train.loc[:, 'vertex_x'] = vertex_xs
train.loc[:, 'vertex_y'] = vertex_ys
train.loc[:, 'bounding_confidence'] = bounding_confidences
train.loc[:, 'bounding_importance'] = bounding_importance_fracs
train.loc[:, 'dominant_blue'] = dominant_blues
train.loc[:, 'dominant_green'] = dominant_greens
train.loc[:, 'dominant_red'] = dominant_reds
train.loc[:, 'dominant_pixel_frac'] = dominant_pixel_fracs
train.loc[:, 'dominant_score'] = dominant_scores
train.loc[:, 'label_description'] = label_descriptions
train.loc[:, 'label_score'] = label_scores


vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
nf_count = 0
nl_count = 0
for pet in test_id:
    try:
        with open('../input/petfinder-adoption-prediction/test_metadata/' + pet + '-1.json', 'r') as f:
            data = json.load(f)
        vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
        vertex_xs.append(vertex_x)
        vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
        vertex_ys.append(vertex_y)
        bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
        bounding_confidences.append(bounding_confidence)
        bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
        bounding_importance_fracs.append(bounding_importance_frac)
        dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
        dominant_blues.append(dominant_blue)
        dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
        dominant_greens.append(dominant_green)
        dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
        dominant_reds.append(dominant_red)
        dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
        dominant_pixel_fracs.append(dominant_pixel_frac)
        dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
        dominant_scores.append(dominant_score)
        if data.get('labelAnnotations'):
            label_description = data['labelAnnotations'][0]['description']
            label_descriptions.append(label_description)
            label_score = data['labelAnnotations'][0]['score']
            label_scores.append(label_score)
        else:
            nl_count += 1
            label_descriptions.append('nothing')
            label_scores.append(-1)
    except FileNotFoundError:
        nf_count += 1
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_descriptions.append('nothing')
        label_scores.append(-1)

print(nf_count)
test.loc[:, 'vertex_x'] = vertex_xs
test.loc[:, 'vertex_y'] = vertex_ys
test.loc[:, 'bounding_confidence'] = bounding_confidences
test.loc[:, 'bounding_importance'] = bounding_importance_fracs
test.loc[:, 'dominant_blue'] = dominant_blues
test.loc[:, 'dominant_green'] = dominant_greens
test.loc[:, 'dominant_red'] = dominant_reds
test.loc[:, 'dominant_pixel_frac'] = dominant_pixel_fracs
test.loc[:, 'dominant_score'] = dominant_scores
test.loc[:, 'label_description'] = label_descriptions
test.loc[:, 'label_score'] = label_scores

14993
0
3948


In [15]:
train_feats=pd.read_csv(r'../input/densenet-pretrain-with-internet-blocked-12b8d1/train_img_features.csv')
test_feats=pd.read_csv(r'../input/densenet-pretrain-with-internet-blocked-12b8d1/test_img_features.csv')

print(train_feats.shape, test_feats.shape)

(14993, 257) (3948, 257)


In [16]:
train_feats.index = train_feats['Unnamed: 0']
train_feats.drop(['Unnamed: 0'],axis=1,inplace=True)
train_feats = train_feats*100
test_feats.index = test_feats['Unnamed: 0']
test_feats.drop(['Unnamed: 0'],axis=1,inplace=True)
test_feats = test_feats*100
print(train.shape, test.shape)
print(train_feats.head())
print(train.head())
n_img_feat = test_feats.shape[1]
train_feats.columns = ["img_feat{}".format(i) for i in range(n_img_feat)]
test_feats.columns = ["img_feat{}".format(i) for i in range(n_img_feat)]

train_feats["PetID"] = train_feats.index
test_feats["PetID"] = test_feats.index

train = pd.merge(train, train_feats, on="PetID")
test = pd.merge(test, test_feats, on="PetID")

print(train.shape, test.shape)

(14993, 158) (3948, 158)
                   0          1          2         3          4         5  \
Unnamed: 0                                                                  
86e1089a3   0.218783  15.961751   3.809780  1.604203   5.749157  0.247455   
6296e909a   0.343610   7.038007   2.715974  2.457085  18.397991  0.253275   
3422e4906   0.304852   5.961684   2.086257  1.777012  12.350991  0.255668   
5842f1ff5   0.258530  19.894393  10.254380  1.501368   8.734525  0.152519   
850a43f90   0.252064  12.662882   9.200954  2.141793  25.643438  0.222140   

                   6         7          8         9     ...             246  \
Unnamed: 0                                              ...                   
86e1089a3   0.601009  0.583615   5.611092  5.324179     ...       63.737607   
6296e909a   0.640988  0.783363  11.660004  5.160271     ...       57.899427   
3422e4906   0.463135  0.318573   8.652384  3.874717     ...       72.400969   
5842f1ff5   0.630636  0.484039   7.38445

In [17]:
train.drop(['Name', 'RescuerID', 'Description','PetID'], axis=1, inplace=True)
test.drop(['Name', 'RescuerID', 'Description','PetID'], axis=1, inplace=True)

In [18]:
numeric_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt', 'AdoptionSpeed', 'doc_sent_mag', 'doc_sent_score',
                'dominant_score', 'dominant_pixel_frac', 'dominant_red', 'dominant_green', 'dominant_blue', 'bounding_importance', 
                'bounding_confidence', 'vertex_x', 'vertex_y','label_score','state_population','state_gdp'
               ]+ ['svd_{}'.format(i) for i in range(120)]+list("img_feat{}".format(i) for i in range(n_img_feat))
#list(range(train_feats.shape[1]))
cat_cols = list(set(train.columns) - set(numeric_cols))
print(len(cat_cols),len(numeric_cols))
train.loc[:, cat_cols] = train[cat_cols].astype('category')
test.loc[:, cat_cols] = test[cat_cols].astype('category')
print(train.shape)
print(test.shape)

# get the categorical features
foo = train.dtypes
cat_feature_names = foo[foo == "category"]
cat_features = [train.columns.get_loc(c) for c in train.columns if c in cat_feature_names]

15 396
(14993, 410)
(3948, 410)


In [46]:
N_SPLITS = 4#5
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = StratifiedKFold(n_splits=N_SPLITS, random_state=42, shuffle=True)
    fold_splits = kf.split(train, target)
    cv_scores = []
    qwk_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0], N_SPLITS))
    all_coefficients = np.zeros((N_SPLITS, 4))
    feature_importance_df = pd.DataFrame()
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/' + str(N_SPLITS))
        if isinstance(train, pd.DataFrame):
            dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        else:
            dev_X, val_X = train[dev_index], train[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y, importances, coefficients, qwk = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        all_coefficients[i-1, :] = coefficients
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            qwk_scores.append(qwk)
            print(label + ' cv score {}: RMSE {} QWK {}'.format(i, cv_score, qwk))
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = train.columns.values
        fold_importance_df['importance'] = importances
        fold_importance_df['fold'] = i
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)        
        i += 1
    print('{} cv RMSE scores : {}'.format(label, cv_scores))
    print('{} cv mean RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv QWK scores : {}'.format(label, qwk_scores))
    print('{} cv mean QWK score : {}'.format(label, np.mean(qwk_scores)))
    print('{} cv std QWK score : {}'.format(label, np.std(qwk_scores)))
    pred_full_test = pred_full_test / float(N_SPLITS)
    results = {'label': label,
               'train': pred_train, 'test': pred_full_test,
                'cv': cv_scores, 'qwk': qwk_scores,
               'qwk_mean':np.mean(qwk_scores),
               'params':params,
               'importance': feature_importance_df,
               'coefficients': all_coefficients}
    return results#['qwk_mean','params']

def runLGB(train_X, train_y, test_X, test_y, test_X2, params):
    print('Prep LGB')
    d_train = lgb.Dataset(train_X, label=train_y)
    d_valid = lgb.Dataset(test_X, label=test_y)
    watchlist = [d_train, d_valid]
    print('Train LGB')
    num_rounds = params.pop('num_rounds')
    verbose_eval = params.pop('verbose_eval')
    early_stop = None
    if params.get('early_stop'):
        early_stop = params.pop('early_stop')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      categorical_feature=list(cat_features),
                      early_stopping_rounds=early_stop)
    
    print('Predict 1/2')
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    optR = OptimizedRounder()
    optR.fit(pred_test_y, test_y)
    coefficients = optR.coefficients()
    pred_test_y_k = optR.predict(pred_test_y, coefficients)
    print("Valid Counts = ", Counter(test_y))
    print("Predicted Counts = ", Counter(pred_test_y_k))
    print("Coefficients = ", coefficients)
    qwk = quadratic_weighted_kappa(test_y, pred_test_y_k)
    print("QWK = ", qwk)
    print('Predict 2/2')
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)
    return pred_test_y.reshape(-1, 1), pred_test_y2.reshape(-1, 1), model.feature_importance(), coefficients, qwk
 
#results = run_cv_model(train, test, target, runLGB, params, rmse, 'lgb')

In [47]:
params = {'application': 'regression', 'bagging_fraction': 0.85, 'boosting': 'gbdt', 'data_random_seed': 17, 
          'early_stop': 100, 'feature_fraction': 0.8, 'lambda_l2': 0.0475, 'learning_rate': 0.015, 'max_depth': 10,
          'metric': 'rmse', 'min_child_samples': 150, 'min_child_weight': 0.02, 'min_split_gain': 0.02, 
          'num_leaves': 60, 'num_rounds': 10000, 'verbose_eval': 100, 'verbosity': 0}
results = run_cv_model(train, test, target, runLGB, params, rmse, 'lgb')
print(results)

Started lgb fold 1/4
Prep LGB
Train LGB


C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 152]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.


KeyboardInterrupt: 

In [29]:
results['qwk_mean']

0.4143325191452145

In [51]:
results_list =[]
params_grid = {'application': ['regression'],
          'boosting': ['gbdt'],
          'metric': ['rmse'],
          'num_leaves': [45,60],#40,50,70],#,80,100
          'max_depth': [10,13,16],#,7
          'learning_rate': [0.012],#2,0.016], #[0.01,0.015,0.025 0.011,0.013
          'bagging_fraction': [0.65,0.75],
          'feature_fraction': [0.65,0.85],
          'min_split_gain': [0.03],#0.02
          'min_child_samples': [70,90],#50,100,200],#150 80,120,170
          'min_child_weight': [0.03],#05,0.04],#0.02
          'lambda_l2': [0.047],#0.0475 0.046,0.049
          'verbosity': [-1],
          'data_random_seed': [17],
          'early_stop': [100],# 600,#200,
          'verbose_eval': [100],
          'num_rounds': [10000]}
with open("tuning"+str(strftime("%d-%m-%H-%M", gmtime()))+".txt", "a") as myfile:
    set_n = 0
    set_n_total = len(list(ParameterGrid(params_grid)))
    print(set_n_total)
    for params in list(ParameterGrid(params_grid)):
        print("Param set N: ",set_n, "out of ", set_n_total)
        print(params)
        results = run_cv_model(train, test, target, runLGB, params, rmse, 'lgb')
        myfile.write(str(results['qwk_mean'])+', ')
        myfile.write(str(results['params'])+'\n')
        set_n+=1


48
Param set N:  0 out of  48
{'application': 'regression', 'bagging_fraction': 0.65, 'boosting': 'gbdt', 'data_random_seed': 17, 'early_stop': 100, 'feature_fraction': 0.65, 'lambda_l2': 0.047, 'learning_rate': 0.012, 'max_depth': 10, 'metric': 'rmse', 'min_child_samples': 70, 'min_child_weight': 0.03, 'min_split_gain': 0.03, 'num_leaves': 45, 'num_rounds': 10000, 'verbose_eval': 100, 'verbosity': -1}
Started lgb fold 1/4
Prep LGB
Train LGB


C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 152]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.02351	valid_1's rmse: 1.09487
[200]	training's rmse: 0.937886	valid_1's rmse: 1.0708
[300]	training's rmse: 0.871927	valid_1's rmse: 1.06043
[400]	training's rmse: 0.815227	valid_1's rmse: 1.05438
[500]	training's rmse: 0.766556	valid_1's rmse: 1.05033
[600]	training's rmse: 0.724075	valid_1's rmse: 1.04801
[700]	training's rmse: 0.685078	valid_1's rmse: 1.04648
[800]	training's rmse: 0.649657	valid_1's rmse: 1.04529
[900]	training's rmse: 0.617328	valid_1's rmse: 1.04445
[1000]	training's rmse: 0.587508	valid_1's rmse: 1.04353
[1100]	training's rmse: 0.559255	valid_1's rmse: 1.04265
[1200]	training's rmse: 0.532061	valid_1's rmse: 1.04212
[1300]	training's rmse: 0.50779	valid_1's rmse: 1.04174
[1400]	training's rmse: 0.48357	valid_1's rmse: 1.04148
[1500]	training's rmse: 0.460627	valid_1's rmse: 1.04094
[1600]	training's rmse: 0.438282	valid_1's rmse: 1.04046
[1700]	training's rmse: 0.418818	valid

C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 152]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.02546	valid_1's rmse: 1.08989
[200]	training's rmse: 0.939403	valid_1's rmse: 1.06452
[300]	training's rmse: 0.872763	valid_1's rmse: 1.05362
[400]	training's rmse: 0.81629	valid_1's rmse: 1.04742
[500]	training's rmse: 0.767503	valid_1's rmse: 1.04354
[600]	training's rmse: 0.725084	valid_1's rmse: 1.04078
[700]	training's rmse: 0.685946	valid_1's rmse: 1.03902
[800]	training's rmse: 0.650517	valid_1's rmse: 1.03812
[900]	training's rmse: 0.618214	valid_1's rmse: 1.03742
[1000]	training's rmse: 0.587168	valid_1's rmse: 1.0368
[1100]	training's rmse: 0.559424	valid_1's rmse: 1.03659
[1200]	training's rmse: 0.532335	valid_1's rmse: 1.03643
[1300]	training's rmse: 0.50682	valid_1's rmse: 1.0363
[1400]	training's rmse: 0.48326	valid_1's rmse: 1.03584
[1500]	training's rmse: 0.46093	valid_1's rmse: 1.03538
[1600]	training's rmse: 0.439719	valid_1's rmse: 1.03527
[1700]	training's rmse: 0.419747	valid_1'

...

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.0022	valid_1's rmse: 1.08443
[200]	training's rmse: 0.900342	valid_1's rmse: 1.06203
[300]	training's rmse: 0.818621	valid_1's rmse: 1.05165
[400]	training's rmse: 0.749446	valid_1's rmse: 1.04612
[500]	training's rmse: 0.688617	valid_1's rmse: 1.04295
[600]	training's rmse: 0.635221	valid_1's rmse: 1.04156
[700]	training's rmse: 0.587265	valid_1's rmse: 1.03984
[800]	training's rmse: 0.544204	valid_1's rmse: 1.0396
[900]	training's rmse: 0.504347	valid_1's rmse: 1.03867
[1000]	training's rmse: 0.468777	valid_1's rmse: 1.03787
[1100]	training's rmse: 0.436028	valid_1's rmse: 1.03734
[1200]	training's rmse: 0.40508	valid_1's rmse: 1.0376
Early stopping, best iteration is:
[1114]	training's rmse: 0.431495	valid_1's rmse: 1.03723
Predict 1/2
Valid Counts =  Counter({4: 1049, 2: 1009, 3: 815, 1: 773, 0: 103})
Predicted Counts =  Counter({2.0: 1390, 3.0: 1036, 4.0: 859, 1.0: 464})
Coefficients =  [0.4978

C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 152]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.00477	valid_1's rmse: 1.07981
[200]	training's rmse: 0.901915	valid_1's rmse: 1.05191
[300]	training's rmse: 0.819984	valid_1's rmse: 1.0394
[400]	training's rmse: 0.750777	valid_1's rmse: 1.03294
[500]	training's rmse: 0.69057	valid_1's rmse: 1.02892
[600]	training's rmse: 0.63699	valid_1's rmse: 1.02709
[700]	training's rmse: 0.588993	valid_1's rmse: 1.02585
[800]	training's rmse: 0.546312	valid_1's rmse: 1.02607
Early stopping, best iteration is:
[704]	training's rmse: 0.587093	valid_1's rmse: 1.02582
Predict 1/2
Valid Counts =  Counter({4: 1049, 2: 1009, 3: 815, 1: 772, 0: 102})
Predicted Counts =  Counter({3.0: 1415, 1.0: 992, 4.0: 781, 2.0: 559})
Coefficients =  [0.43809655 2.15432596 2.35023753 2.96668432]
QWK =  0.46865063808557883
Predict 2/2
lgb cv score 3: RMSE 1.0258157877984349 QWK 0.46865063808557883
Started lgb fold 4/4
Prep LGB
Train LGB


C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 152]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\vinograd\Anaconda5.3\envs\tensorflow_gpuenv\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.00243	valid_1's rmse: 1.08532
[200]	training's rmse: 0.900182	valid_1's rmse: 1.06176
[300]	training's rmse: 0.818527	valid_1's rmse: 1.05105
[400]	training's rmse: 0.748681	valid_1's rmse: 1.04598
[500]	training's rmse: 0.687609	valid_1's rmse: 1.04308
[600]	training's rmse: 0.633774	valid_1's rmse: 1.04037
[700]	training's rmse: 0.585682	valid_1's rmse: 1.03916
[800]	training's rmse: 0.542312	valid_1's rmse: 1.03842
[900]	training's rmse: 0.502664	valid_1's rmse: 1.03832
Early stopping, best iteration is:
[828]	training's rmse: 0.530946	valid_1's rmse: 1.0382
Predict 1/2
Valid Counts =  Counter({4: 1049, 2: 1009, 3: 814, 1: 772, 0: 102})
Predicted Counts =  Counter({3.0: 1246, 2.0: 1196, 4.0: 662, 1.0: 642})
Coefficients =  [0.49362882 1.98461964 2.44558353 3.03275006]
QWK =  0.4483090310744081
Predict 2/2
lgb cv score 4: RMSE 1.0381960293503978 QWK 0.4483090310744081
lgb cv RMSE scores : [1.04260

In [44]:
with open("tuning"+str(strftime("%d-%m-%H-%M", gmtime()))+".txt", "a") as myfile:
    myfile.write(str(results['qwk_mean'])+', ')
    myfile.write(str(results['params'])+'\n')


In [37]:
import datetime
str(datetime.datetime.now())

'2019-02-11 11:32:27.676603'

In [40]:
from time import gmtime, strftime
strftime("%d-%m-%H-%M", gmtime())

'11-02-10:34'

In [19]:
imports = results['importance'].groupby('feature')['feature', 'importance'].mean().reset_index()
imports.sort_values('importance', ascending=False)

,feature,importance
1,Breed1,915.8
0,Age,427.4
2,Breed2,284.4
14,State,207.6
13,Quantity,206.2
409,vertex_y,198.6
285,label_score,164.4
401,svd_93,161.2
332,svd_30,158.2
168,img_feat224,154.0


from sklearn.model_selection import ParameterGrid
param_grid = {'a': [1, 2], 'b': [True, False]}
for par in list(ParameterGrid(param_grid)):
    print(par)

NL_East = {'Phillies': [645, 469], 'Braves': [599, 548], 'Mets': [653, 672]}
for team in NL_East.keys():
    print(team)
for team, runs in NL_East.items():
    print(team,runs)

In [20]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
train_predictions = [r[0] for r in results['train']]
train_predictions = optR.predict(train_predictions, coefficients_).astype(int)
Counter(train_predictions)

[0.51438792 1.77228447 2.50440234 2.91016939]


Counter({2: 3973, 1: 3063, 3: 3024, 4: 4232, 0: 701})

In [21]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
test_predictions = [r[0] for r in results['test']]
test_predictions = optR.predict(test_predictions, coefficients_).astype(int)
Counter(test_predictions)

[0.51438792 1.77228447 2.50440234 2.91016939]


Counter({2: 1031, 3: 796, 4: 1146, 1: 844, 0: 131})

In [22]:
print("True Distribution:")
print(pd.value_counts(target, normalize=True).sort_index())
print("Test Predicted Distribution:")
print(pd.value_counts(test_predictions, normalize=True).sort_index())
print("Train Predicted Distribution:")
print(pd.value_counts(train_predictions, normalize=True).sort_index())


True Distribution:
0    0.027346
1    0.206096
2    0.269259
3    0.217368
4    0.279931
Name: AdoptionSpeed, dtype: float64
Test Predicted Distribution:
0    0.033181
1    0.213779
2    0.261145
3    0.201621
4    0.290274
dtype: float64
Train Predicted Distribution:
0    0.046755
1    0.204295
2    0.264990
3    0.201694
4    0.282265
dtype: float64


In [23]:
pd.DataFrame(sk_cmatrix(target, train_predictions), index=list(range(5)), columns=list(range(5)))

,0,1,2,3,4
0,69,127,93,72,49
1,331,1065,932,474,288
2,205,1019,1309,803,701
3,80,609,983,800,787
4,16,243,656,875,2407


In [24]:
print(quadratic_weighted_kappa(target, train_predictions))
print(rmse(target, [r[0] for r in results['train']]))
submission = pd.DataFrame({'PetID': test_id, 'AdoptionSpeed': test_predictions})
submission.head()

0.44635777194152937
1.0380527894123763


,PetID,AdoptionSpeed
0,378fcc4fc,2
1,73c10e136,3
2,72000c4c5,4
3,e147a4b9f,4
4,43fbba852,4


In [25]:
submission.to_csv('submission.csv', index=False)